In [ ]:
# Importando bibliotecas
import requests
import json
import pandas as pd
import time
import os
from datetime import datetime

from dotenv import load_dotenv

load_dotenv()

TINY_TOKEN = os.getenv("TINY_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")

# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": 'localhost',
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}


pd.set_option('display.max_columns', None)

# Registra o tempo antes da execução
start_prog = time.time()

# Configurações da API Tiny

token = TINY_TOKEN
formato = 'JSON'

In [ ]:
# Carregando IDs de produtos

excel_path = r"..\Data\Base\Envios Full.xlsx"  # Caminho para o arquivo Excel

sheet_name = "Relação Full x Tiny"  # Nome da planilha

col1 = "ID Tiny"  # Nome da coluna a carregar
col2 = "Título do anúncio"

# Use o Pandas para ler a coluna específica
df_tiny_id_original = pd.read_excel(excel_path, sheet_name=sheet_name, usecols=[col1, col2])

# Salve o DataFrame original
df_tiny_id = df_tiny_id_original.copy()

# Agora, df conterá apenas a coluna especificada
df_tiny_id.shape

In [ ]:
# # Converte para números inteiros e substitui os não numéricos por NaN
# df_tiny_id['ID Tiny'] = pd.to_numeric(df_tiny_id['ID Tiny'], errors='coerce')

# df_tiny_id.shape

In [ ]:
# Remova as linhas com valores NaN - Linha 378
df_tiny_id_no_nan = df_tiny_id.dropna(subset=['ID Tiny'])

df_tiny_id_no_nan.shape

In [ ]:
# # Salve o DataFrame original
# df_tiny_id_original = df_tiny_id.copy()

# # Remova as linhas com valores NaN
# df_tiny_id_no_nan = df_tiny_id.dropna(subset=['ID Tiny'])

# # Compare os DataFrames para encontrar a linha removida
# linha_removida = df_tiny_id_original[~df_tiny_id_original.index.isin(df_tiny_id_no_nan.index)]

# # Exiba a linha removida
# print(linha_removida)


In [ ]:
# Verifica se há duplicatas
duplicatas = df_tiny_id_no_nan.duplicated()
numero_de_duplicatas = duplicatas.sum()

print(f'Número de duplicatas: {numero_de_duplicatas}')

# # Encontre as duplicatas no DataFrame
# duplicatas = df_tiny_id_no_nan[df_tiny_id_no_nan.duplicated(keep=False)]

# # Exiba as próprias duplicatas
# duplicatas

In [ ]:
# Remove as duplicatas e atualiza o DataFrame
df_tiny_id_no_dup = df_tiny_id_no_nan.drop_duplicates()

df_tiny_id_no_dup.shape

In [ ]:
# Requisitando produtos à API Tiny

start_time = time.time() # Registra o tempo antes da execução

url = 'https://api.tiny.com.br/api2/produto.obter.php'

# df_tiny_id_no_dup = df_tiny_id_no_dup.head(3)
#df_tiny_id = pd.read_csv('tiny_id_no_dup.csv')
df_tiny_id = df_tiny_id_no_dup

responses = []  # Lista para armazenar os resultados
num = 0

def enviarREST(url, data, optional_headers=None):
    headers = optional_headers if optional_headers is not None else {}

    response = requests.post(url, data=data, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Problema com {url}, Status Code: {response.status_code}")

    return response.text


for id in df_tiny_id['ID Tiny']:
    print(f'Buscando dados de {id}')
    num +=1
    print(f'Loop: {num}')
    data = {'token': token, 'id': id, 'formato': formato}
    response = enviarREST(url, data)
    responses.append(response)

    # Verifica se é múltiplo de 50 para aguardar 1min a cada 50 requisições
    if num % 50 == 0:
        print("Esperando 1 minuto...")
        time.sleep(60)  # Pausa por 1 minuto

# Registra o tempo depois da execução
end_time = time.time()

# Calcula o tempo decorrido
elapsed_time = end_time - start_time

In [ ]:
print(f"Tempo :decorrido {elapsed_time / 60} minutos")
print(f"Total de resultados encontrados: {len(responses)}")

In [ ]:
responses

In [ ]:
# Gravando respostas em txt
datetime_now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

file_path = f"../Data/Output/tiny_products_json_{datetime_now}.txt"

with open(file_path, 'w') as file:
    for line in responses:
        file.write(line + '\n')  # Adicione uma quebra de linha

print(f'Os dados foram gravados no arquivo: {file_path}')

In [ ]:
# Verificandos e há erros de processamento
text_search1 = '"status_processamento":"1"'
text_search2 = '"status_processamento":"2"'

lines_search1 = []
lines_search2 = []

with open(file_path, 'r') as file:
    for n_line, line in enumerate(file, start=1):
        if text_search1 in line:
            lines_search1.append(n_line)
        if text_search2 in line:
            lines_search2.append(n_line)

if lines_search1 and lines_search2:
    print(f'"status_processamento":"1" foi encontrado nas linhas: {lines_search1}')
    print(f'"status_processamento":"2" foi encontrado nas linhas: {lines_search2}')
elif lines_search1:
    print(f'"status_processamento":"1" foi encontrado nas linhas: {lines_search1}')
    print('"status_processamento":"2" não foi encontrado em nenhuma linha.')
elif lines_search2:
    print(f'"status_processamento":"2" foi encontrado nas linhas: {lines_search2}')
    print('"status_processamento":"1" não foi encontrado em nenhuma linha.')
else:
    print('Não foi verificado erro de processamento"')


# Df Base

In [42]:
json_list = responses

tiny_prod_df = pd.DataFrame() # Inicialize o DataFrame vazio

for json_str in json_list:
    json_data = json.loads(json_str) # Transforme a string JSON em um objeto Python

    # Extrair a parte "produto" do JSON
    produto = json_data["retorno"]["produto"]
    # print(produto)
    
    # Adicione os dados extraídos ao DataFrame
    tiny_prod_df = pd.concat([tiny_prod_df, pd.DataFrame([produto])], ignore_index=True)

# Exibir o DataFrame
tiny_prod_df.head(1)


,id,nome,codigo,unidade,preco,preco_promocional,ncm,origem,gtin,gtin_embalagem,localizacao,peso_liquido,peso_bruto,estoque_minimo,estoque_maximo,id_fornecedor,nome_fornecedor,codigo_fornecedor,codigo_pelo_fornecedor,unidade_por_caixa,preco_custo,preco_custo_medio,situacao,tipo,classe_ipi,valor_ipi_fixo,cod_lista_servicos,descricao_complementar,garantia,cest,obs,tipoVariacao,variacoes,idProdutoPai,sob_encomenda,dias_preparacao,marca,tipoEmbalagem,alturaEmbalagem,comprimentoEmbalagem,larguraEmbalagem,diametroEmbalagem,qtd_volumes,categoria,anexos,imagens_externas,classe_produto,seo_title,seo_keywords,link_video,seo_description,slug
0,509517168,Cabo P10 X P10 Santo Angelo Ninja 10ft - 3.05m,7899028808537,Un,46.9,0,8544.42.00,2,7899028808537,,Torre Liverpool 2/ B4-3 25.06.2021,0,0.5,0,0,660366502,SMA CABOS E SISTEMAS LTDA,,4.5.20.232.121,,25.45,25.45,A,P,,0.0000,,,6,,,N,,0,N,0,,2,3.0,23.0,18.0,0.0,0,Instrumentos Musicais >> Instrumentos de Corda...,[],[],S,,,,,


In [51]:
print(tiny_prod_df.shape)

remove_cols = ['anexos', 'imagens_externas']

tiny_prod_df = tiny_prod_df.drop(columns=remove_cols)

print(tiny_prod_df.shape)

(717, 52)
(717, 50)


In [52]:
# Verifica se há duplicatas
duplicatas = tiny_prod_df.duplicated()
total_duplicatas = duplicatas.sum() 

print(f'Número de duplicatas: {total_duplicatas}')

# Encontra duplicatas no DataFrame
duplicatas = tiny_prod_df[tiny_prod_df.duplicated(keep=False)]

# Exiba duplicatas
duplicatas['id']

l_duplicatas = duplicatas[duplicatas['id'] == '565663877']
l_duplicatas.shape

Número de duplicatas: 235


(44, 50)

In [53]:
# Cria DataFrame booleano para verificar se cada célula contém uma lista
is_list_df = tiny_prod_df.applymap(lambda x: isinstance(x, list))

# Verifica se há alguma coluna que contenha pelo menos uma lista
colunas_com_listas = is_list_df.any()

# Exibe colunas que contêm listas
colunas_com_listas = colunas_com_listas[colunas_com_listas]
colunas_com_listas

C:\Users\couti\AppData\Local\Temp\ipykernel_17476\1738426329.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  is_list_df = tiny_prod_df.applymap(lambda x: isinstance(x, list))


Series([], dtype: bool)

In [64]:
# Remove as duplicatas e atualiza o DataFrame
tiny_prod_no_dup = tiny_prod_df.drop_duplicates()

print(tiny_prod_df.shape)
print(tiny_prod_no_dup.shape)

(717, 50)
(482, 50)


# Salvando na base de dados

In [68]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(**db_config)

cursor = conn.cursor()
n = 1

# Itere sobre as linhas do DataFrame e insira os dados na tabela "tiny_products"
for index, row in tiny_prod_no_dup.iterrows():
    print(f'Loop nº: {n}')
    query = """
    INSERT INTO tiny_products (tiny_id,nome,sku_tiny,unidade,preco,ncm,origem,gtin,peso_bruto,estoque_minimo,estoque_maximo,id_fornecedor,nome_fornecedor,
    codigo_pelo_fornecedor,preco_custo,preco_custo_medio,situacao,tipo,cest,marca,tipo_embalagem,altura_embalagem,comprimento_embalagem,largura_embalagem,
    diametro_embalagem,qtd_volumes,categoria)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);
    """
    values = (
        row['id'],
        row['nome'],
        row['codigo'],
        row['unidade'],
        row['preco'],
        row['ncm'],
        row['origem'],
        row['gtin'],
        row['peso_bruto'],
        row['estoque_minimo'],
        row['estoque_maximo'],
        row['id_fornecedor'],
        row['nome_fornecedor'],
        row['codigo_pelo_fornecedor'],
        row['preco_custo'],
        row['preco_custo_medio'],
        row['situacao'],
        row['tipo'],
        row['cest'],
        row['marca'],
        row['tipoEmbalagem'],
        row['alturaEmbalagem'],
        row['comprimentoEmbalagem'],
        row['larguraEmbalagem'],
        row['diametroEmbalagem'],
        row['qtd_volumes'],
        row['categoria']
    )
    # print(values)
    n += 1
    cursor.execute(query, values)

# Faça o commit para salvar as alterações no banco de dados
conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()
print('Produtos adicionados ao Banco de dados.')


Loop nº: 1


UniqueViolation: duplicate key value violates unique constraint "tiny_products_pkey"
DETAIL:  Key (tiny_id)=(509517168) already exists.


In [69]:
t = 'Instrumentos Musicais >> Instrumentos de Corda >> Acessórios >> Cordas e Encordoamentos >> Para Violões e Guitarras'
len(t)

115